In [1]:
import zfit
import math
from zfit import z
import numpy as np
import tensorflow as tf
from scipy.optimize import minimize

zfit.settings.options['numerical_grad'] = True
class HistPDF(zfit.pdf.BasePDF):

    def __init__(self, hist_args, hist_bins, obs, name='HistPDF'):
        self.rv_hist = scipy.stats.rv_histogram([hist_args, hist_bins])
        super().__init__(obs=obs, name=name)

    def _unnormalized_pdf(self, x):
        x = z.unstack_x(x)
        probs =  z.py_function(func=self.rv_hist.pdf, inp=[x], Tout=tf.float64)
        probs.set_shape(x.shape)
        return probs

/anaconda3/lib/python3.6/site-packages/zfit/util/execution.py:70: UserWarning: Not running on Linux. Determining available cpus for thread can failand be overestimated. Workaround (only if too many cpus are used):`zfit.run.set_n_cpu(your_cpu_number)`
  warnings.warn("Not running on Linux. Determining available cpus for thread can fail"


In [2]:
# mu2 = zfit.Parameter("mu2", 5., step_size=0)
# sigma2 = zfit.Parameter("sigma2", 1., step_size=0)
# lambd2 = zfit.Parameter("lambda2", -0.2, step_size=0)
# frac2 = zfit.Parameter("fraction2", 0.5, 0, 1)
# frac1 = zfit.Parameter("fraction1", 0.5, step_size=0)
# create space
obs1 = zfit.Space("x", limits=(0, 10))
obs2 = zfit.Space("x", limits=(0, 10))

# parameters
mu1 = zfit.Parameter("mu1", 5., 1, 10, step_size=0)
sigma1 = zfit.Parameter("sigma1", 1., 0.1, 10, step_size=0)
lambd1 = zfit.Parameter("lambda1", -0.2, -1, -0.01, step_size=0)
frac1 = zfit.Parameter("fraction1", 0.5, 0, 1)

mu2 = zfit.Parameter("mu2", 5., step_size=0)
sigma2 = zfit.Parameter("sigma2", 1., step_size=0)
lambd2 = zfit.Parameter("lambda2", -0.2, step_size=0)
frac2 = zfit.Parameter("fraction2", 0.5, step_size=0)




gauss1 = zfit.pdf.Gauss(mu=mu1, sigma=sigma1, obs=obs1)
exponential1 = zfit.pdf.Exponential(lambd1, obs=obs1)
model1 = zfit.pdf.SumPDF([gauss1, exponential1], fracs=frac1)


gauss2 = zfit.pdf.Gauss(mu=mu2, sigma=sigma2, obs=obs2)
exponential2 = zfit.pdf.Exponential(lambd2, obs=obs2)
model2 = zfit.pdf.SumPDF([gauss2, exponential2], fracs=frac2)

In [3]:
n_sample = 10000

exp_data = exponential2.sample(n=n_sample * (1 - frac1)).numpy()

gauss_data = gauss2.sample(n=n_sample * frac1).numpy()

data = model1.create_sampler(n_sample, limits=obs1)
data.resample()

In [4]:
data_np = data[:, 0].numpy()
exp_data_np = exp_data[:, 0]
gauss_data_np = gauss_data[:, 0]

In [5]:
data_hist = np.histogram(data_np, bins=30)
exp_data_hist = np.histogram(exp_data_np, bins=30)
gauss_data_hist = np.histogram(gauss_data_np, bins=30)
sim_hists = []
sim_hists.append(exp_data_hist)
sim_hists.append(gauss_data_hist)

In [89]:
import scipy
class FractionFitterV2:

    def __init__(self, data_hist, sim_hists, P):
        self.data_hist = data_hist
        self.P = np.array(P)  # vectorization 3
        self.sim_hists = [hist for hist in sim_hists]
        self.d = np.array(self.data_hist[0]) # where d[i] amount of events in bin from data
        self.N_D = np.sum(self.data_hist[0])#all observable data amount

        # vectorization 3
        self.N = np.array([np.sum(h[0]) for h in sim_hists])# amount of simulation data from sources e.g. N[0] from source 0 .. N[j] from source j
        self.sources_num = len(P)
        self.bins_num = len(data_hist[0])
        
    def norma(self, v):
        return math.sqrt(np.sum(v ** 2))
    #function to minimize for finding optimat t according to (15) from the paper        
    def sq_f(self, t, a, p, i):
        return (np.sum((p * a[:, i] / (1 + p * t))) - self.d[i]/(1 - t))**2 
    
    def sq_f_vectorized(self, t, a, p, i):
        term_1 = np.sum((p[:, None] * a[:, i] / (1 + p[:, None] * t[None, :])))
        term_2 = self.d[i]/(1 - t)
        return (term_1 - term_2)**2
    
    def f(self, t, a, p, i):
        return np.sum((p * a[:, i] / (1 + p * t))) - self.d[i]/(1 - t)

    def f_vectorized(self, t, a, p, i):  # add an axis argumend to sum
        term1 = np.sum((p[:, None] * a[:, i] / (1 + p[:, None] * t[None, :])),
                       axis=0)
        term2 = self.d[i]/(1 - t)
        return term1 - term2

    def der_fsq(self, t, a, p, i):
        return -2 * self.f(t, a, p, i) * (np.sum((p * a[:, i])*(p/(1 + p * t)**2)) + self.d[i]/(1 - t)**2)

    def der_f(self, t, a, p, i):  # TODO + self.d[i]/(1 - t)**2
        return (np.sum((p * a[:, i])*(p/(1 + p * t)**2)) + self.d[i]/(1 - t)**2)
    
    def der_f_vectorized(self, t, a, p, i):
        return np.sum((p[:, None] * a[:, i] * p[:, None])/(1 + p[:, None] * t[None, :])**2, axis=0) + self.d[i]/(1 - t)**2
    
    def sqF(self, p, A): # need to find optimal step for p, p_(k+1) = p_k - k * sqF/div_sqF
        res = 0
        for j in range(self.sources_num):
            tmp_res = 0
            for i in range(self.bins_num):
                tmp_res += ((self.d[i] * A[j][i])/np.sum(p * A[:, i]) - A[j][i])
            res += tmp_res**2
            
        return res
                
    def div_sqF(self, p, k, A):
        res = 0
        for j in range(self.sources_num):
            sum1 = 0
            sum2 = 0
            for i in range(self.bins_num):
                sum1 -= (self.d[i] * A[j][i] * A[k][i])/(np.sum(p * A[:, i]))**2
                sum2 += ((self.d[i] * A[j][i])/np.sum(p * A[:, i]) - A[j][i])
            res += sum1*sum2
        return res
            

    def fit(self, eps):
        # let assume initial set of p_j:
        # p = []
        # p_new = []
        # for i in range(self.sources_num):
        #     p.append(self.N_D * self.P[i]/self.N[i])
        
        p = self.N_D * self.P / self.N  # speedup technical 3:
        p_new = np.array(p).copy()

        # technical 4
        zeros_shape = (self.sources_num, self.bins_num)
        a = np.zeros(zeros_shape)  #a[j][i] amount of observations in i bin from j source
        for j in range(self.sources_num):
            for i in range(self.bins_num):
                a[j][i] = self.sim_hists[j][0][i]

        nonzero_indices = np.where(self.d != 0)[0]
        

        while True: 

            # t = []# t[i] = 1 - d[i]/f[i]
            # t calculating ...
            # for i in range(self.bins_num):
            #     if(self.d[i] == 0):
            #         t.append(1)
            #         continue

            #     t.append(scipy.optimize.root(self.f, x0=0.1, 
            #                                  args=(a, p, i),
            #                                  method='hybr',  # 'krylov',
            #                                  tol=None,
            #                                  callback=None,
            #                                  options={}).x[0])
            
            t = np.ones_like(self.d)
            x0 = t[nonzero_indices] * 0.1
            p = np.asarray(p)
            # print(self.d.shape)
            
            # print(nonzero_indices.shape)
            t_solved = scipy.optimize.root(self.f_vectorized,
                                           x0=x0, 
                                        #    x0=0.1 * np.ones_like(nonzero_indices), 
                                             args=(a, p, nonzero_indices),
                                             jac=self.der_f_vectorized,
                                             method='hybr',  # 'krylov',
                                             tol=None,
                                             callback=None,
                                             options={}).x
            t[nonzero_indices] = t_solved

            # print("-1/max(p)= ", -1/max(p))
            # print(t)
            A = np.zeros(self.sources_num)  #A[j][i] fitted amount of observations in i bin from j source

            # Techical 5
            A = a/(1 + p[j, None]*t[None, i])
            A = np.maximum(A, 0.1)
            # for j in range(self.sources_num):
            #     for i in range(self.bins_num):
            #         A[j][i] = a[j][i]/(1 + p[j]*t[i])
            #         if(A[j][i] == 0.0):
            #             A[j][i] = 0.1
                    
            # print("p=", p)
            #bounds on sum of p = 1 and p > 0
            #
            
            for i in range(len(p)):
                p_new[i] = p[i] - self.sqF(p, A)/self.div_sqF(p, i, A)
                
            print(np.abs(self.norma(p_new) - self.norma(p)))
            # print(p_new)
            # print(p)
            p_old = p
            p = p_new.copy()  # bugfix 2
            if np.abs(self.norma(p) - self.norma(p_old)) < eps:
                return p
            


In [90]:
fitter = FractionFitterV2(data_hist=data_hist, sim_hists=sim_hists, P=[0.4, 0.6])

In [91]:
p = fitter.fit(1e-1)
print(p)

TypeError: fsolve: there is a mismatch between the input and output shape of the 'fprime' argument 'der_f_vectorized'.Shape should be (30, 30) but it is (30,).

In [92]:
p

NameError: name 'p' is not defined

In [ ]:
P = []
for j in range(len(p)):
    P.append(p[j] * fitter.N[j]/fitter.N_D)

In [ ]:
P

In [ ]:
zfit.run.set_autograd_mode(False)
zfit.run.set_graph_mode(False)

# create our favorite minimizer
minimizer = zfit.minimize.IpyoptV1()


# minimizer = zfit.minimize.Minuit()
# minimizer = zfit.minimize.ScipyTrustKrylovV1()
# minimizer = zfit.minimize.NLoptLBFGSV1()


def func(x):
    x = np.array(x)  # make sure it's an array
    return np.sum((x - 0.1) ** 2 + x[1] ** 4)


# we can also use a more complicated function instead
# from scipy.optimize import rosen as func


# we need to set the errordef, the definition of "1 sigma"
func.errordef = 0.5

# initial parameters
params = [1, -3, 2, 1.4, 11]
# or for a more fine-grained control
# params = {
#     'value': [1, -3, 2, 1.4, 11],  # mandatory
#     'lower': [-2, -5, -5, -10, -15],  # lower bound, can be omitted
#     'upper': [2, 4, 5, 10, 15],  # upper bound, can be omitted
#     'step_size': [0.1] * 5,  # initial step size, can be omitted
# }

# minimize
result = minimizer.minimize(func, params)

# estimate errors
result.hesse()
result.errors()
print(result)

In [ ]:
print(np.sum((P[:] * A[:,0])))

In [ ]:
print(A[:, :])